In [3]:
import pandas as py
import numpy as np
from scipy.integrate import solve_ivp
from scipy.integrate import odeint
import plotly.graph_objects as go


In [7]:
# Define the differential equations
def dI_dt(I, E, lambda_E, lambda_R):
    return lambda_E * E - lambda_R * I

def dE_dt(S, E, lambda_S, f_q):
    return lambda_S * S * f_q - lambda_E * E

def dS_dt(S, I, lambda_S, lambda_R, f_q):
    return -lambda_S * S * f_q + lambda_R * I

# Define the function for the probability f(q(t)) (uniform distribution)
def f_q(K, M):
    return K / (M + K)

# Function to integrate the differential equations
def simulate_virus_spread(S0, E0, I0, lambda_S, lambda_E, lambda_R, M, total_time, dt):
    num_steps = int(total_time / dt) + 1
    t = np.linspace(0, total_time, num_steps)

    # Initial conditions
    y0 = [S0, E0, I0]

    # Parameters
    args = (lambda_S, lambda_E, lambda_R, M)

    # Define the ODE system
    def system(y, t, *args):
        S, E, I = y
        lambda_S, lambda_E, lambda_R, M = args
        fq = f_q(I, M)
        dS = dS_dt(S, I, lambda_S, lambda_R, fq)
        dE = dE_dt(S, E, lambda_S, fq)
        dI = dI_dt(I, E, lambda_E, lambda_R)  # Corrected call to dI_dt
        return [dS, dE, dI]

    # Integrate the ODE system
    solution = odeint(system, y0, t, args=args)

    return t, solution[:, 0], solution[:, 1], solution[:, 2]




In [10]:
# Parameters
S0 = 1000  # Initial number of susceptible peers
E0 = 10    # Initial number of exposed peers
I0 = 1     # Initial number of infected peers
lambda_S = 0.1  # Rate of downloading new files per minute
lambda_E = 0.05 # Rate of becoming infected per minute
lambda_R = 0.01 # Rate of recovery per minute
M = 1000    # Number of uninfected files
total_time = 1200   # Total simulation time (in minutes)
dt = 0.1    # Time step for integration

# Simulate virus spread
t, S, E, I = simulate_virus_spread(S0, E0, I0, lambda_S, lambda_E, lambda_R, M, total_time, dt)

# Plotting using Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=t, y=S, mode='lines', name='Susceptible'))
fig.add_trace(go.Scatter(x=t, y=E, mode='lines', name='Exposed'))
fig.add_trace(go.Scatter(x=t, y=I, mode='lines', name='Infected'))

fig.update_layout(
    title='Simulation of Virus Spread in a P2P Network',
    xaxis_title='Time (minutes)',
    yaxis_title='Population',
    template='plotly_white'
)

fig.show()